## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Warm-up: a simple 2d system

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from copy import copy
from pydrake.all import MathematicalProgram, Solve, Polynomial, Variables
matplotlib.rcParams['figure.figsize'] = (8, 8)

In [ ]:
def f(x):
    return [-x[0] - 2 * x[1]**2, -x[1] - x[0] * x[1] - 2 * x[1]**3]

def find_lyapunov(f, n, d):
    
    # Initialize program and optimization variables.
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(n, 'x')
    
    # Lyapunov conditions.
    v = prog.NewSosPolynomial(Variables(x), d)[0].ToExpression()
    prog.AddLinearConstraint(v.Substitute({x[0]: 0, x[1]: 0}) == 0)
    prog.AddLinearConstraint(v.Substitute({x[0]: 1, x[1]: 0}) == 1)
    vdot = v.Jacobian(x).dot(f(x))
    prog.AddSosConstraint(-vdot)
    
    # Solve SOS and print solution.
    result = Solve(prog)
    assert result.is_success()
    v = Polynomial(result.GetSolution(v))
    
    return v, x

In [ ]:
def plot_f_and_v(f, v, x, x1lim=(-1, 1), x2lim=(-1, 1), n=300j, **kwargs):
    
    # Evaluate f.
    X1, X2 = np.mgrid[x1lim[0]:x1lim[1]:n, x2lim[0]:x2lim[1]:n]
    X1d, X2d = f([X1, X2])

    # Color the streamlines according to the magnitude of f(x).
    color = np.sqrt(X1d**2 + X2d**2)

    # Vector field.
    strm = plt.streamplot(X1.T[0], X2[0], X1d.T, X2d.T, color=color.T)
    
    # Evaluate v.
    V = copy(X1)
    for i in range(X1.shape[0]):
        for j in range(X1.shape[1]):
            V[i, j] = v.Evaluate({x[0]: X1[i, j], x[1]: X2[i, j]})
            
    # Plot v.
    plt.contour(X1, X2, V, colors='r', **kwargs)
    
    # Misc plot settings.
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.xlim(x1lim)
    plt.ylim(x2lim)
    plt.gca().set_aspect('equal')
    plt.title(r'Lyapunov function (red), vector field (colors)')

In [ ]:
# Find Lyapunov function and plot it.
n = 2
deg = 2
v, x = find_lyapunov(f, n, deg)
plot_f_and_v(f, v, x)

# Nasty high-degree polynomial system
Ahmadi, Parrilo - "Converse Results on Existence of Sum of Squares Lyapunov Functions"

In [ ]:
# High-degree dynamics.
def f(x):
    return [
    - .15*x[0]**7 + 200*x[0]**6*x[1] - 10.5*x[0]**5*x[1]**2 - 807*x[0]**4*x[1]**3 \
    + 14*x[0]**3*x[1]**4 + 600*x[0]**2*x[1]**5 - 3.5*x[0]*x[1]**6 + 9*x[1]**7,
    - 9*x[0]**7 - 3.5*x[0]**6*x[1] - 600*x[0]**5*x[1]**2 + 14*x[0]**4*x[1]**3 \
    + 807*x[0]**3*x[1]**4 - 10.5*x[0]**2*x[1]**5 - 200*x[0]*x[1]**6 + .15*x[1]**7
]

# Find Lyapunov function and plot it.
n = 2
deg = 8
v, x = find_lyapunov(f, n, deg)
plot_f_and_v(f, v, x, levels=[10.**i for i in range(-5, 2)])